<a href="https://colab.research.google.com/github/galenos-project/literature-mining/blob/main/Copy_of_LSR_DRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymed

## PubMed search for literature: Depression, Anxiety, Psychosis AND Psychadelics AND published between 2014-2024

In [ ]:
#PubMed Search string and results

from pymed import PubMed
import pandas as pd

pubmed = PubMed(tool="PubMedSearcher", email="avahomiar@gmail.com")

search_term = "(((mood) OR (depress*) OR (affective disorder)) OR ((psychosis) OR (schizo*) OR (psychotic))) AND ((psychedelic OR hallucinogen OR entheogen OR hallucinogenic OR psychotropic)) AND (2014/2024[Date - Publication])"
results = pubmed.query(search_term)
articleList = []
articleInfo = []

for article in results:
    articleDict = article.toDict()
    articleList.append(articleDict)

for article in articleList:
    pubmedId = article['pubmed_id'].partition('\n')[0]
    articleInfo.append({
        u'pubmed_id': pubmedId,
        u'title': article['title'],
        u'journal': article['journal'],
        u'abstract': article['abstract'],})

    # keywords available
    if 'keywords' in article:
        articleInfo[-1]['keywords'] = article['keywords']

articlesPD = pd.DataFrame.from_dict(articleInfo)
export_csv = articlesPD.to_csv(r'C:\Users\ahomiar\OneDrive - Nexus365\Desktop\export_dataframe.csv', index=None, header=True)

print(articlesPD.head())


  pubmed_id                                              title  \
0  27793241  Seizure due to multiple drugs intoxication: a ...   
1  25529756  Efficacy of ranitidine in olanzapine-induced w...   
2  25148542  Characterization and evaluation of self-nanoem...   
3  27136907  Pharmacokinetic profile after multiple deltoid...   
4  26706484  Pregabalin Treatment of a Patient With Complex...   

                                          journal  \
0  Brazilian journal of anesthesiology (Elsevier)   
1                Early intervention in psychiatry   
2                                   Drug delivery   
3       Clinical pharmacology in drug development   
4                             Pediatric neurology   

                                            abstract  \
0  The mechanism of the antidepressant effect of ...   
1  Weight gain has long been recognized as a side...   
2  The purpose of this work was to develop self-n...   
3  Paliperidone palmitate (PP) is a once-monthly ...   
4  C

##Using articles from search results to combine all sections

In [ ]:
for article in results:
    articleDict = article.toDict()

    # Combine title, abstract, and keywords into one text
    text = f"{articleDict['title']}\n{articleDict.get('abstract', '')}\n{' '.join(articleDict.get('keywords', []))}"

    articleDict['text'] = text
    articleList.append(articleDict)

articlesPD = pd.DataFrame.from_dict(articleList)
export_csv = articlesPD.to_csv(r'C:\Users\ahomiar\OneDrive - Nexus365\Desktop\export_dataframe.csv', index=None, header=True)

print(articlesPD)


                                            pubmed_id  \
0                                            27793241   
1                                            25529756   
2                                            25148542   
3                                            27136907   
4                                            26706484   
..                                                ...   
95  25628381\n17981266\n21123312\n21677641\n164699...   
96                                           25626189   
97                                           25624181   
98                                           25619678   
99                                           25619432   

                                                title  \
0   Seizure due to multiple drugs intoxication: a ...   
1   Efficacy of ranitidine in olanzapine-induced w...   
2   Characterization and evaluation of self-nanoem...   
3   Pharmacokinetic profile after multiple deltoid...   
4   Pregabalin Treatment of a 

##Combining all sections of all articles to create one text --> corpus

In [ ]:
search_term = "(((mood) OR (depress*) OR (affective disorder)) OR ((psychosis) OR (schizo*) OR (psychotic))) AND ((psychedelic OR hallucinogen OR entheogen OR hallucinogenic OR psychotropic)) AND (2014/2024[Date - Publication])"
results = pubmed.query(search_term)
articleList = []
articleInfo = []
corpus = ""

for article in results:
    articleDict = article.toDict()

    # Combine title, abstract, and keywords into one text
    text = f"{articleDict['title']}\n{articleDict.get('abstract', '')}\n{' '.join(articleDict.get('keywords', []))}"

    # Append text to corpus
    corpus += text + "\n"

    articleDict['text'] = text
    articleList.append(articleDict)

articlesPD = pd.DataFrame.from_dict(articleList)
export_csv = articlesPD.to_csv(r'C:\Users\ahomiar\OneDrive - Nexus365\Desktop\export_dataframe.csv', index=None, header=True)

print(corpus)


Seizure due to multiple drugs intoxication: a case report.
The mechanism of the antidepressant effect of bupropion is not fully understood. Besides, using it in the treatment of depression, it is found to be effective in reducing withdrawal symptoms due to smoking cessation. A 28-year-old female patient with a history of depression was admitted to emergency department an hour after ingestion of bupropion, quetiapine, and levothyroxine in high doses to commit suicide. While accepting her into the Intensive Care Unit, she was awake, alert, disoriented and agitated. After 2h, the patient had a generalized tonic-clonic seizure. The necessary treatment was given and 9h later with hemodynamic improvement, the patients' mental status improved. Bupropion may cause unusual behaviors such as delusions, paranoia, hallucinations, or confusion. The risk of seizure is strongly dose-dependent. We want to emphasize the importance of early gastric lavage and administration of activated charcoal.
Buprip

##Preprocessing the corpus and applying Latent Semantic Analysis (LSA)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

# Define preprocessing function
def preprocess_text(text):

# Implement preprocessing steps
  return text

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(doc) for doc in corpus.split('\n')]

# Create TF-IDF matrix
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(preprocessed_corpus)

# Apply Singular Value Decomposition (SVD)
n_topics = 50  # Number of topics to extract
svd = TruncatedSVD(n_topics)
lsa = make_pipeline(svd, Normalizer(copy=False))
lsa_matrix = lsa.fit_transform(tfidf_matrix)

# Extract topics (words = 5)
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-6:-1]])
    print(f"Topic {i}: {top_terms}")


Topic 1: treatment, depression, patients, antidepressant, schizophrenia
Topic 2: la, en, que, les, los
Topic 3: receptor, antidepressant, effects, effect, mice
Topic 4: schizophrenia, olanzapine, patients, dyskinesia, tardive
Topic 5: schizophrenia, response, disorder, major, depressive
Topic 6: olanzapine, depression, fluoxetine, bipolar, receptor
Topic 7: schizophrenia, resistant, self, treatment, paliperidone
Topic 8: tablets, magnesium, imipramine, fatigue, release
Topic 9: tablets, olanzapine, magnesium, imipramine, bipolar
Topic 10: affective, bipolar, disorder, staging, illness
Topic 11: self, efficacy, induced, disorder, los
Topic 12: receptor, psychotropic, tablets, sigma, los
Topic 13: self, receptor, sigma, stigma, disorder
Topic 14: les, des, et, psychotropic, une
Topic 15: antipsychotic, weight, sigma, drug, gain
Topic 16: response, cognitive, effect, psychomotor, depression
Topic 17: response, affective, staging, illness, unipolar
Topic 18: depression, psychiatric, follow

### LSA - each topic 10 words

In [ ]:
# Extract topics (words = 10)
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-11:-1]])
    print(f"Topic {i}: {top_terms}")

Topic 1: treatment, depression, patients, antidepressant, schizophrenia, disorder, depressive, study, response, outcome
Topic 2: la, en, que, les, los, des, et, para, une, se
Topic 3: receptor, antidepressant, effects, effect, mice, test, modafinil, nmda, like, activity
Topic 4: schizophrenia, olanzapine, patients, dyskinesia, tardive, weight, study, fatigue, gain, ranitidine
Topic 5: schizophrenia, response, disorder, major, depressive, biomarkers, imaging, predict, paliperidone, palmitate
Topic 6: olanzapine, depression, fluoxetine, bipolar, receptor, combination, treatment, resistant, test, gain
Topic 7: schizophrenia, resistant, self, treatment, paliperidone, palmitate, depression, physicians, refractory, resident
Topic 8: tablets, magnesium, imipramine, fatigue, release, syndrome, substances, duloxetine, effect, pain
Topic 9: tablets, olanzapine, magnesium, imipramine, bipolar, release, substances, study, disorder, gain
Topic 10: affective, bipolar, disorder, staging, illness, uni

###LSA - each topic 20 words

In [ ]:
# Extract topics (words = 20)
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-21:-1]])
    print(f"Topic {i}: {top_terms}")

Topic 1: treatment, depression, patients, antidepressant, schizophrenia, disorder, depressive, study, response, outcome, self, resistant, predict, clinical, effects, bipolar, major, cognitive, effect, medication
Topic 2: la, en, que, les, los, des, et, para, une, se, le, el, pour, traitement, una, las, ou, dans, sont, au
Topic 3: receptor, antidepressant, effects, effect, mice, test, modafinil, nmda, like, activity, sigma, antagonist, glutamate, induced, study, receptors, dopaminergic, citalopram, trkb, involvement
Topic 4: schizophrenia, olanzapine, patients, dyskinesia, tardive, weight, study, fatigue, gain, ranitidine, efficacy, 20, treated, paliperidone, group, assess, follow, chronic, duloxetine, incidence
Topic 5: schizophrenia, response, disorder, major, depressive, biomarkers, imaging, predict, paliperidone, palmitate, predictors, resonance, magnetic, antipsychotic, receptor, antipsychotics, psychosis, outcome, genetic, remission
Topic 6: olanzapine, depression, fluoxetine, bip

##Preprocessing the text and applying Non-negative Matrix Factorization (NMF)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

# Define preprocessing function
def preprocess_text(text):
# Implement preprocessing steps
    return text

# Preprocess corpus
preprocessed_corpus = [preprocess_text(doc) for doc in corpus.split('\n')]

# Create TF-IDF matrix
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(preprocessed_corpus)

# Apply Non-negative Matrix Factorization (NMF)
n_topics = 50  # Number of topics to extract
nmf = NMF(n_components=n_topics, random_state=1)
nmf_matrix = nmf.fit_transform(tfidf_matrix)

# Extract topics
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(nmf.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-6:-1]])
    print(f"Topic {i}: {top_terms}")

Topic 1: treatment, depression, physicians, resident, resistant
Topic 2: la, los, en, que, para
Topic 3: nmda, citalopram, activity, fluoxetine, swim
Topic 4: dyskinesia, tardive, olanzapine, incidence, schizophrenia
Topic 5: disorder, major, biomarkers, depressive, treatment
Topic 6: bipolar, combination, fluoxetine, olanzapine, disorder
Topic 7: fatigue, duloxetine, syndrome, chronic, inventory
Topic 8: tablets, imipramine, magnesium, substances, release
Topic 9: gain, ranitidine, weight, olanzapine, 300
Topic 10: affective, staging, illness, unipolar, resistant
Topic 11: remission, patients, depressed, speed, mdd
Topic 12: psychotropic, children, adolescents, japan, 2002
Topic 13: stigma, self, anxiety, level, stigmatization
Topic 14: les, des, la, et, une
Topic 15: group, groups, significant, 05, waist
Topic 16: cognitive, deficits, depression, vortioxetine, pr
Topic 17: response, author, parikh, antipsychotics, predictors
Topic 18: days, follow, 30, subsequent, utilization
Topic 1

###NMF - each topic 10 words

In [ ]:
# Extract topics (words = 10)
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-11:-1]])
    print(f"Topic {i}: {top_terms}")

Topic 1: treatment, depression, patients, antidepressant, schizophrenia, disorder, depressive, study, response, outcome
Topic 2: la, en, que, les, los, des, et, para, une, se
Topic 3: receptor, antidepressant, effects, effect, mice, test, modafinil, nmda, like, activity
Topic 4: schizophrenia, olanzapine, patients, dyskinesia, tardive, weight, study, fatigue, gain, ranitidine
Topic 5: schizophrenia, response, disorder, major, depressive, biomarkers, imaging, predict, paliperidone, palmitate
Topic 6: olanzapine, depression, fluoxetine, bipolar, receptor, combination, treatment, resistant, test, gain
Topic 7: schizophrenia, resistant, self, treatment, paliperidone, palmitate, depression, physicians, refractory, resident
Topic 8: tablets, magnesium, imipramine, fatigue, release, syndrome, substances, duloxetine, effect, pain
Topic 9: tablets, olanzapine, magnesium, imipramine, bipolar, release, substances, study, disorder, gain
Topic 10: affective, bipolar, disorder, staging, illness, uni

###NMF - each topic 20 words

In [ ]:
# Extract topics
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-21:-1]])
    print(f"Topic {i}: {top_terms}")

Topic 1: treatment, depression, patients, antidepressant, schizophrenia, disorder, depressive, study, response, outcome, self, resistant, predict, clinical, effects, bipolar, major, cognitive, effect, medication
Topic 2: la, en, que, les, los, des, et, para, une, se, le, el, pour, traitement, una, las, ou, dans, sont, au
Topic 3: receptor, antidepressant, effects, effect, mice, test, modafinil, nmda, like, activity, sigma, antagonist, glutamate, induced, study, receptors, dopaminergic, citalopram, trkb, involvement
Topic 4: schizophrenia, olanzapine, patients, dyskinesia, tardive, weight, study, fatigue, gain, ranitidine, efficacy, 20, treated, paliperidone, group, assess, follow, chronic, duloxetine, incidence
Topic 5: schizophrenia, response, disorder, major, depressive, biomarkers, imaging, predict, paliperidone, palmitate, predictors, resonance, magnetic, antipsychotic, receptor, antipsychotics, psychosis, outcome, genetic, remission
Topic 6: olanzapine, depression, fluoxetine, bip

##Preprocessing text and applying Latent Dirichlet Allocation (LDA)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Define preprocessing function
def preprocess_text(text):

    # Implement preprocessing steps
    return text

# Preprocess corpus
preprocessed_corpus = [preprocess_text(doc) for doc in corpus.split('\n')]

# Create CountVectorizer to convert text into a matrix of token counts
vectorizer = CountVectorizer(stop_words='english')
doc_term_matrix = vectorizer.fit_transform(preprocessed_corpus)

# Define number of topics
num_topics = 50

# Apply LDA
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(doc_term_matrix)

# Print topics generated by LDA model
terms = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_terms = ", ".join([terms[i] for i in topic.argsort()[:-6:-1]])
    print(f"Topic {topic_idx + 1}: {top_terms}")

Topic 1: antidepressant, social, canada, france, inequalities
Topic 2: non, baseline, significant, patients, responders
Topic 3: lps, effects, induced, behavior, depression
Topic 4: treatment, mri, used, clinical, alterations
Topic 5: 95, ci, use, antidepressants, pour
Topic 6: effects, patients, care, depression, collaborative
Topic 7: swn, 20, items, prototypical, depressive
Topic 8: major, tablets, episodes, caffeine, pourrait
Topic 9: tasks, depression, copying, neuropsychological, major
Topic 10: level, concentration, consciousness, effets, phenobarbital
Topic 11: treatment, antipsychotic, patient, medication, kyna
Topic 12: fatigue, syndrome, treatment, pain, duloxetine
Topic 13: release, sustained, snedds, ziprasidone, kg
Topic 14: groups, treatment, group, 05, 30
Topic 15: receptor, depression, effect, antagonist, test
Topic 16: la, que, los, una, se
Topic 17: psychomotor, controlled, trial, randomized, effects
Topic 18: depression, cognitive, participants, measures, effect
Top

###LDA - each topic with 10 words

In [ ]:
 # Extract topics (words = 10)
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-11:-1]])
    print(f"Topic {i}: {top_terms}")

Topic 1: treatment, depression, patients, antidepressant, schizophrenia, disorder, depressive, study, response, outcome
Topic 2: la, en, que, les, los, des, et, para, une, se
Topic 3: receptor, antidepressant, effects, effect, mice, test, modafinil, nmda, like, activity
Topic 4: schizophrenia, olanzapine, patients, dyskinesia, tardive, weight, study, fatigue, gain, ranitidine
Topic 5: schizophrenia, response, disorder, major, depressive, biomarkers, imaging, predict, paliperidone, palmitate
Topic 6: olanzapine, depression, fluoxetine, bipolar, receptor, combination, treatment, resistant, test, gain
Topic 7: schizophrenia, resistant, self, treatment, paliperidone, palmitate, depression, physicians, refractory, resident
Topic 8: tablets, magnesium, imipramine, fatigue, release, syndrome, substances, duloxetine, effect, pain
Topic 9: tablets, olanzapine, magnesium, imipramine, bipolar, release, substances, study, disorder, gain
Topic 10: affective, bipolar, disorder, staging, illness, uni

###LDA - each topic with 20 words

In [ ]:
# Extract topics
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_, start=1):
    top_terms = ", ".join([terms[index] for index in topic.argsort()[:-21:-1]])
    print(f"Topic {i}: {top_terms}")

Topic 1: treatment, depression, patients, antidepressant, schizophrenia, disorder, depressive, study, response, outcome, self, resistant, predict, clinical, effects, bipolar, major, cognitive, effect, medication
Topic 2: la, en, que, les, los, des, et, para, une, se, le, el, pour, traitement, una, las, ou, dans, sont, au
Topic 3: receptor, antidepressant, effects, effect, mice, test, modafinil, nmda, like, activity, sigma, antagonist, glutamate, induced, study, receptors, dopaminergic, citalopram, trkb, involvement
Topic 4: schizophrenia, olanzapine, patients, dyskinesia, tardive, weight, study, fatigue, gain, ranitidine, efficacy, 20, treated, paliperidone, group, assess, follow, chronic, duloxetine, incidence
Topic 5: schizophrenia, response, disorder, major, depressive, biomarkers, imaging, predict, paliperidone, palmitate, predictors, resonance, magnetic, antipsychotic, receptor, antipsychotics, psychosis, outcome, genetic, remission
Topic 6: olanzapine, depression, fluoxetine, bip

##Preprocess the text and apply Semantic Signal Separation (with and without publication year filter)


In [ ]:
!pip install turftopic


In [ ]:
from turftopic import SemanticSignalSeparation


In [ ]:
# Define preprocessing function
def preprocess_text(text):
    # Implement preprocessing steps
    return text

# Collect articles
search_term = "(((mood) OR (depress*) OR (affective disorder)) OR ((psychosis) OR (schizo*) OR (psychotic))) AND ((psychedelic OR hallucinogen OR entheogen OR hallucinogenic OR psychotropic)) AND (2014/2024[Date - Publication])"
results = pubmed.query(search_term)
corpus = ""

# Collect article texts
for article in results:
    articleDict = article.toDict()

    # Combine title, abstract, and keywords into one text
    text = f"{articleDict['title']}\n{articleDict.get('abstract', '')}\n{' '.join(articleDict.get('keywords', []))}"

    # Append text to corpus
    corpus += text + "\n"

# Preprocess corpus
preprocessed_corpus = [preprocess_text(doc) for doc in corpus.split('\n')]

# Initialize and fit the SemanticSignalSeparation model
model = SemanticSignalSeparation(10, encoder="all-MiniLM-L12-v2")
model.fit(preprocessed_corpus)

# Print topics
model.print_topics()


Output()

[15:49:24] Documents encoded.                                                                          ]8;id=584295;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=599730;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#75\75]8;;\

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, 
whiten='unit-variance' will be used by default.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not 
converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(

[15:49:25] Decomposition done.                                                                         ]8;id=841980;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=761181;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#78\78]8;;\

           Term extraction done.                                                                       ]8;id=35453;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=306147;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#81\81]8;;\

           Vocabulary encoded.                                                                         ]8;id=905733;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=338190;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#84\84]8;;\

           Model fitting done.                                                                         ]8;id=927982;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=303772;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#88\88]8;;\

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic ID ┃ Highest Ranking                                   ┃ Lowest Ranking                                   ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        0 │ schizophrenia, psychotropic, brain, psychiatric,  │ remission, weight, results, measures, outcome    │
│          │ antipsychotic                                     │                                                  │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        1 │ bipolar, mood, cognitive, psychiatric, brain      │ associated, medications, treatments, weight,     │
│          │                                                   │ aripiprazole                                     │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        2 │ measures, cognitive, compared, data, scale        │ antipsychotics, aripiprazole, olanzapine,        │
│          │                                                   │ symptoms, antipsychotic                          │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        3 │ remission, mood, severe, symptoms, depression     │ antipsychotics, schizophrenia, antipsychotic,    │
│          │                                                   │ olanzapine, predict                              │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        4 │ antidepressant, antidepressants, receptors,       │ patients, patient, clinical, illness, treated    │
│          │ fluoxetine, receptor                              │                                                  │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        5 │ antidepressants, antidepressant, medications,     │ brain, mice, cognitive, receptors, receptor      │
│          │ prescription, depressed                           │                                                  │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        6 │ 50, 10, 12, 20, years                             │ patient, medications, therapeutic, anxiety,      │
│          │                                                   │ medication                                       │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        7 │ age, weight, health, risk, higher                 │ treatment, treatments, treated, remission,       │
│          │                                                   │ therapeutic                                      │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        8 │ bipolar, olanzapine, antipsychotics,              │ depressive, research, efficacy, outcomes,        │
│          │ aripiprazole, medications                         │ studies                                          │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        9 │ efficacy, use, strategies, effective, using       │ disorders, disorder, syndrome, disease, illness  │
└──────────┴───────────────────────────────────────────────────┴──────────────────────────────────────────────────┘

In [ ]:
# Define preprocessing function
def preprocess_text(text):
    # Implement preprocessing steps
    return text

# Collect articles
search_term = "(((mood) OR (depress*) OR (affective disorder)) OR ((psychosis) OR (schizo*) OR (psychotic))) AND ((psychedelic OR hallucinogen OR entheogen OR hallucinogenic OR psychotropic))"
results = pubmed.query(search_term)
corpus = ""

# Collect article texts
for article in results:
    articleDict = article.toDict()

    # Combine title, abstract, and keywords into one text
    text = f"{articleDict['title']}\n{articleDict.get('abstract', '')}\n{' '.join(articleDict.get('keywords', []))}"

    # Append text to the corpus
    corpus += text + "\n"

# Preprocess the corpus
preprocessed_corpus = [preprocess_text(doc) for doc in corpus.split('\n')]

# Initialize and fit the SemanticSignalSeparation model
model = SemanticSignalSeparation(10, encoder="all-MiniLM-L12-v2")
model.fit(preprocessed_corpus)

# Print topics
model.print_topics()



Output()

[15:50:24] Documents encoded.                                                                          ]8;id=952364;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=936030;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#75\75]8;;\

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, 
whiten='unit-variance' will be used by default.
  warnings.warn(

[15:50:25] Decomposition done.                                                                         ]8;id=767901;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=708984;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#78\78]8;;\

           Term extraction done.                                                                       ]8;id=469394;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=683331;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#81\81]8;;\

[15:50:26] Vocabulary encoded.                                                                         ]8;id=858084;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=261558;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#84\84]8;;\

           Model fitting done.                                                                         ]8;id=605273;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=68954;file:///usr/local/lib/python3.10/dist-packages/turftopic/models/decomp.py#88\88]8;;\

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic ID ┃ Highest Ranking                                  ┃ Lowest Ranking                                    ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        0 │ depression, depressive, mood, antidepressant,    │ cortex, valproate, brain, autism, children        │
│          │ remission                                        │                                                   │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        1 │ benzodiazepines, anxiety, medications,           │ psilocybin, brain, remission, dose, receptor      │
│          │ antidepressants, medication                      │                                                   │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        2 │ valproate, ketamine, prescribing, remission,     │ brain, cortex, psychiatric, schizophrenia,        │
│          │ esketamine                                       │ psychotropic                                      │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        3 │ antidepressants, benzodiazepines,                │ patients, clinicians, treated, treating, patient  │
│          │ antidepressant, serotonin, sleep                 │                                                   │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        4 │ covid, care, depressive, autism, bipolar         │ psychedelic, psychedelics, ayahuasca, lsd,        │
│          │                                                  │ psilocybin                                        │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        5 │ antipsychotics, clozapine, schizophrenia,        │ sleep, experience, interventions, performed, pain │
│          │ psychotic, psilocybin                            │                                                   │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        6 │ syndrome, symptoms, illness, disorders,          │ efficacy, research, interventions, sample,        │
│          │ psychosis                                        │ antidepressant                                    │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        7 │ benzodiazepines, prescribing, antipsychotics,    │ understanding, time, present, post, long          │
│          │ pharmacological, prevalence                      │                                                   │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        8 │ self, blood, development, cortex, related        │ antipsychotics, clozapine, medications,           │
│          │                                                  │ treatment, prescribing                            │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        9 │ esketamine, ketamine, therapeutic, therapy,      │ 20, 10, 12, years, 30                             │
│          │ serotonin                                        │                                                   │
└──────────┴──────────────────────────────────────────────────┴───────────────────────────────────────────────────┘